# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [4]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [6]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Reinforcement learning (RL) is a powerful machine learning paradigm used in various domains. Here are some notable use cases:

1. **Robotics**:
   - **Autonomous Navigation**: RL helps robots learn to navigate complex environments, avoiding obstacles and optimizing paths.
   - **Manipulation Tasks**: Robots learn to perform tasks like picking, placing, and assembling objects.

2. **Gaming**:
   - **Game Playing**: RL has been used to train agents to play games like chess, Go, and video games, often surpassing human performance.
   - **Game Design**: It helps in creating adaptive and intelligent non-player characters (NPCs).

3. **Finance**:
   - **Trading Strategies**: RL can optimize trading strategies by learning from market data to maximize returns.
   - **Portfolio Management**: It helps in dynamic asset allocation by adapting to market changes.

4. **Healthcare**:
   - **Personalized Medicine**: RL can tailor treatment plans by learning from patient data.
   - **Resource Management**: Optimizing hospital operations, such as scheduling and resource allocation.

5. **Autonomous Vehicles**:
   - **Driving Policies**: RL is used to develop driving policies for self-driving cars, enabling them to make real-time decisions.
   - **Traffic Management**: It helps in optimizing traffic flow and reducing congestion.

6. **Natural Language Processing**:
   - **Dialogue Systems**: RL improves conversational agents by optimizing responses based on user interactions.
   - **Text Summarization**: It helps in creating concise summaries by learning from feedback on summary quality.

7. **Energy Management**:
   - **Smart Grids**: RL optimizes energy distribution and consumption in smart grids.
   - **Renewable Energy Systems**: It aids in managing and optimizing the use of renewable energy sources.

8. **Industrial Automation**:
   - **Process Optimization**: RL is used to optimize manufacturing processes and reduce waste.
   - **Predictive Maintenance**: It predicts equipment failures and schedules maintenance to minimize downtime.

9. **Marketing and Advertising**:
   - **Recommendation Systems**: RL personalizes recommendations by learning user preferences over time.
   - **Ad Placement**: It optimizes ad placement and bidding strategies to increase engagement.

10. **Telecommunications**:
    - **Network Optimization**: RL optimizes network traffic and resource allocation to improve service quality.
    - **Fault Detection**: It helps in identifying and mitigating network issues.

These use cases demonstrate RL's versatility in solving complex decision-making problems across various industries.

In [7]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

I'm unable to recall previous interactions. Could you remind me what your question was?

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [10]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

You asked for some use cases for reinforcement learning.

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](./images/memory_diagram.png)

In [11]:
index1_name = "srch-index-usecases"
index2_name = "srch-index-csv"
index3_name = "srch-index-books"
indexes = [index1_name, index2_name, index3_name]

In [12]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)

If you check closely in prompts.py, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. Now it is the time to use it. It is basically a place holder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.

**Now let's add memory to it:**

In [13]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [14]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [15]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [16]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))


Reinforcement learning (RL) can be applied to various use cases across different industries. Here are a few examples:

1. **Digital Factory Use Cases**: RL can be used to create digital twins of field instruments, valves, and intelligent electronic devices. It helps in simulating control systems topologies and industrial automation systems, enabling standardized information exchange and achieving real-time visibility into globally distributed assets [[1]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

2. **Automation and Process Optimization**: AutomationML in conjunction with RL can interconnect various engineering tools across disciplines such as plant planning, mechanical engineering, and process control. This integration facilitates efficient configuration and communication between tools, reducing errors and improving data consistency [[2]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

3. **Asset Management**: RL can optimize asset management by predicting maintenance needs, monitoring device health, and automating device calibration. It aids in lifecycle management and ensures that devices operate within their optimal range [[3]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

4. **Cybersecurity**: In cybersecurity, RL can be used to enhance security measures by identifying potential threats and vulnerabilities. It can support multifactor authentication and enforce password policies, thereby mitigating risks [[4]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

These use cases highlight how reinforcement learning can drive efficiency, improve safety, and optimize operations in various sectors.

In [17]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [18]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! If you have more questions in the future, feel free to ask. Goodbye!

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [19]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [20]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [21]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [22]:
config

{'configurable': {'session_id': 'session465', 'user_id': 'user336'}}

In [23]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning (RL) is applied in various domains to solve complex problems. Here are some use cases:

1. **Digital Factory Applications**: In the context of a digital factory, RL can be used to optimize processes and operations. It can help in creating digital twins of control systems and topologies, allowing for real-time adjustments and improvements in efficiency [[1]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

2. **Automation and Control Systems**: RL can be used to develop and refine control systems in industrial automation. By simulating various scenarios, RL algorithms can learn optimal strategies for controlling machinery and devices, leading to better resource management and reduced downtime [[1]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

3. **Predictive Maintenance**: RL can be applied in predictive maintenance to anticipate equipment failures and optimize maintenance schedules. This ensures that maintenance is performed only when necessary, reducing costs and preventing unexpected breakdowns [[1]](https://blobstorageffpanhhmq7wy3.blob.core.windows.net/usecases/TR2258%20Digital%20Factory%20-%20Information%20and%20Exchange%20Models.PDF).

These examples demonstrate how reinforcement learning can be effectively utilized in industrial settings to enhance operations, reduce costs, and improve system reliability.

In [24]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [25]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

We discussed use cases of reinforcement learning, focusing on applications in digital factories and industrial automation.

In [26]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

You're welcome! If you have any more questions, feel free to ask.

In [27]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

You asked for a one line summary of our conversation, so I provided a concise recap of the discussion. If you need more details, feel free to ask!

In [28]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

I aimed to keep it concise as per your request for a one line summary. If you'd like a longer summary, just let me know!

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.